# Bar in NYC, Richmond VA, Seattle
Beer, fresh out of college drinks
- microbrew
- need fine options on food menu
- shitty vodka is best vodka

- microwinery <3 

_Target customers_<br>
- Middle-aged
- Mid-20s hipsters

Restaurant
SPORTS BAR that is always closed

# USE BASE DATA

First step is to get requests:

In [39]:
import requests
import json
import sys
import pandas as pd
print(sys.path)

['/Users/Mike/Flatiron_020121/yelp_phase1_project', '/opt/anaconda3/lib/python38.zip', '/opt/anaconda3/lib/python3.8', '/opt/anaconda3/lib/python3.8/lib-dynload', '', '/opt/anaconda3/lib/python3.8/site-packages', '/opt/anaconda3/lib/python3.8/site-packages/aeosa', '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/Mike/.ipython']


In [5]:
url =  'https://api.yelp.com/v3/businesses/search' #points to url from my own yelp developer page

In [6]:
from  keys  import  client_id, api_key

In [8]:
print(api_key)

fTHz2Imsswcpx8-QlwgX0o8nzr1_4hzTzAoP2Q30Ox0Z_Ijd6XxxxUBaHsIDKnXB3z6XsK2RliQ16vc__xa6KZcummrlsvbi52Ssc2y4hUt9F-afkZxq1LNh5ZomYHYx


In [9]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


In [10]:
### THIS CAN BE DYNAMIC
# what type of business do you want to search
term = 'Thai' # this would 

#where do you want to perform this search
location = 'Queens'

# what is your other parameter you want to search against
categories = 'restaurants'

In [11]:

url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
    
            }


In [12]:
url_params

{'term': 'Thai',
 'location': 'Queens',
 'categories': 'restaurants',
 'limit': 50}

In [13]:
response = requests.get(url, headers=headers, params=url_params)

In [14]:
print(response)

<Response [200]>

In [15]:
response.text 

'{"businesses": [{"id": "cegA4jf16vEt7NCQ7cpu2w", "alias": "kind-fresh-meadows", "name": "Kin\'d", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/3rnKMO-RGXBAVYXAgffmwQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/kind-fresh-meadows?adjust_creative=vGqQbkXzrgSl7z2qib6lAg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vGqQbkXzrgSl7z2qib6lAg", "review_count": 767, "categories": [{"alias": "thai", "title": "Thai"}], "rating": 4.5, "coordinates": {"latitude": 40.73031, "longitude": -73.77814}, "transactions": ["delivery", "pickup"], "price": "$$", "location": {"address1": "192-03 Union Tpke", "address2": "", "address3": null, "city": "Fresh Meadows", "zip_code": "11366", "country": "US", "state": "NY", "display_address": ["192-03 Union Tpke", "Fresh Meadows, NY 11366"]}, "phone": "+17184680888", "display_phone": "(718) 468-0888", "distance": 6273.193942408963}, {"id": "1E7lCNaRAM1Da0vkt5rcvw", "alias": "thai-mission-maspeth", "name": "Thai Miss

In [16]:
json.loads(response.text)

{'businesses': [{'id': 'cegA4jf16vEt7NCQ7cpu2w',
   'alias': 'kind-fresh-meadows',
   'name': "Kin'd",
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/3rnKMO-RGXBAVYXAgffmwQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/kind-fresh-meadows?adjust_creative=vGqQbkXzrgSl7z2qib6lAg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vGqQbkXzrgSl7z2qib6lAg',
   'review_count': 767,
   'categories': [{'alias': 'thai', 'title': 'Thai'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.73031, 'longitude': -73.77814},
   'transactions': ['delivery', 'pickup'],
   'price': '$$',
   'location': {'address1': '192-03 Union Tpke',
    'address2': '',
    'address3': None,
    'city': 'Fresh Meadows',
    'zip_code': '11366',
    'country': 'US',
    'state': 'NY',
    'display_address': ['192-03 Union Tpke', 'Fresh Meadows, NY 11366']},
   'phone': '+17184680888',
   'display_phone': '(718) 468-0888',
   'distance': 6273.193942408963},
  {'id': '1E7

In [17]:
response.json()

{'businesses': [{'id': 'cegA4jf16vEt7NCQ7cpu2w',
   'alias': 'kind-fresh-meadows',
   'name': "Kin'd",
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/3rnKMO-RGXBAVYXAgffmwQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/kind-fresh-meadows?adjust_creative=vGqQbkXzrgSl7z2qib6lAg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vGqQbkXzrgSl7z2qib6lAg',
   'review_count': 767,
   'categories': [{'alias': 'thai', 'title': 'Thai'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.73031, 'longitude': -73.77814},
   'transactions': ['delivery', 'pickup'],
   'price': '$$',
   'location': {'address1': '192-03 Union Tpke',
    'address2': '',
    'address3': None,
    'city': 'Fresh Meadows',
    'zip_code': '11366',
    'country': 'US',
    'state': 'NY',
    'display_address': ['192-03 Union Tpke', 'Fresh Meadows, NY 11366']},
   'phone': '+17184680888',
   'display_phone': '(718) 468-0888',
   'distance': 6273.193942408963},
  {'id': '1E7

In [18]:
data = response.json()

In [19]:
data.keys()

dict_keys(['businesses', 'total', 'region'])

In [20]:
data['total']

2600

In [21]:
def yelp_call(counter): 
    response = requests.get(url, headers=headers, params=url_params) # our url, header and params should be consistent, atleast with our Yelp data
    return response.json()

In [22]:
yelp_call(0)

{'businesses': [{'id': 'cegA4jf16vEt7NCQ7cpu2w',
   'alias': 'kind-fresh-meadows',
   'name': "Kin'd",
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/3rnKMO-RGXBAVYXAgffmwQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/kind-fresh-meadows?adjust_creative=vGqQbkXzrgSl7z2qib6lAg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vGqQbkXzrgSl7z2qib6lAg',
   'review_count': 767,
   'categories': [{'alias': 'thai', 'title': 'Thai'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.73031, 'longitude': -73.77814},
   'transactions': ['pickup', 'delivery'],
   'price': '$$',
   'location': {'address1': '192-03 Union Tpke',
    'address2': '',
    'address3': None,
    'city': 'Fresh Meadows',
    'zip_code': '11366',
    'country': 'US',
    'state': 'NY',
    'display_address': ['192-03 Union Tpke', 'Fresh Meadows, NY 11366']},
   'phone': '+17184680888',
   'display_phone': '(718) 468-0888',
   'distance': 6273.193942408963},
  {'id': '1E7

#### Parsing the Data

In [23]:
business = data['businesses'][0]

In [25]:
business
# name, rating, review_count, price, ##location?

{'id': 'cegA4jf16vEt7NCQ7cpu2w',
 'alias': 'kind-fresh-meadows',
 'name': "Kin'd",
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/3rnKMO-RGXBAVYXAgffmwQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/kind-fresh-meadows?adjust_creative=vGqQbkXzrgSl7z2qib6lAg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vGqQbkXzrgSl7z2qib6lAg',
 'review_count': 767,
 'categories': [{'alias': 'thai', 'title': 'Thai'}],
 'rating': 4.5,
 'coordinates': {'latitude': 40.73031, 'longitude': -73.77814},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '192-03 Union Tpke',
  'address2': '',
  'address3': None,
  'city': 'Fresh Meadows',
  'zip_code': '11366',
  'country': 'US',
  'state': 'NY',
  'display_address': ['192-03 Union Tpke', 'Fresh Meadows, NY 11366']},
 'phone': '+17184680888',
 'display_phone': '(718) 468-0888',
 'distance': 6273.193942408963}

In [26]:
business['name']

"Kin'd"

In [27]:
business['location']#['address1']

{'address1': '192-03 Union Tpke',
 'address2': '',
 'address3': None,
 'city': 'Fresh Meadows',
 'zip_code': '11366',
 'country': 'US',
 'state': 'NY',
 'display_address': ['192-03 Union Tpke', 'Fresh Meadows, NY 11366']}

In [28]:
full_address = business['location']['address1'], business['location']['city'], business['location']['state'], business['location']['zip_code']
full_address

('192-03 Union Tpke', 'Fresh Meadows', 'NY', '11366')

In [29]:
business_rating = business['rating']
business_rating

4.5

In [30]:
biz_tuple = (business['name'], business['location']['display_address'], business['rating'])

In [31]:
biz_tuple

("Kin'd", ['192-03 Union Tpke', 'Fresh Meadows, NY 11366'], 4.5)

In [32]:
def parse_data(list_of_data):
    # create a container to hold our parsed data
    businesses = []
    # loop through our business and 
    for business in list_of_data:
    # parse each individual business into a tuple
        biz_tuple = (business['name'],
                     business['location']['display_address'], 
                     business['rating'])
    # add each individual business tuple to our data container
        businesses.append(biz_tuple)
    # return the container with all of the parsed results
    return businesses

In [33]:
parsed_results = parse_data(data['businesses'])

In [38]:
parsed_results # list of tuples representing name, address, and rating of establishments

[("Kin'd", ['192-03 Union Tpke', 'Fresh Meadows, NY 11366'], 4.5),
 ('Thai Mission', ['70-11 Grand Ave', 'Maspeth, NY 11378'], 5.0),
 ('Bangkok Cuisine', ['10718 70th Rd', 'Forest Hills, NY 11375'], 4.0),
 ('Sup Thai Kitchen', ['178-19 Union Tpke', 'Fresh Meadows, NY 11366'], 4.5),
 ('Nur Thai', ['63-32 Woodhaven Blvd', 'Rego Park, NY 11374'], 4.0),
 ('Kitchen 79', ['37-70 79th St', 'Jackson Heights, NY 11372'], 4.5),
 ('Ayada Thai', ['7708 Woodside Ave', 'Elmhurst, NY 11373'], 4.0),
 ('Ma Kin', ['7419 Metropolitan Ave', 'Middle Village, NY 11379'], 4.5),
 ('SriPraPhai Thai Restaurant', ['64-13 39th Ave', 'Woodside, NY 11377'], 4.0),
 ('Pye Boat Noodle', ['35-13 Broadway', 'Astoria, NY 11106'], 4.5),
 ('Heng Thai', ['23-06 Jackson Ave', 'Long Island City, NY 11101'], 5.0),
 ('Grace Thai Restaurant', ['5617 Junction Blvd', 'Elmhurst, NY 11373'], 4.5),
 ('BKNY Thai Restaurant',
  ['47-11 Francis Lewis Blvd', 'Bayside, NY 11361'],
  4.0),
 ('Dek Sen', ['8608 Whitney Ave', 'Elmhurst, NY 11

In [36]:
pd.DataFrame(parsed_results)

,0,1,2
0,Kin'd,"[192-03 Union Tpke, Fresh Meadows, NY 11366]",4.5
1,Thai Mission,"[70-11 Grand Ave, Maspeth, NY 11378]",5.0
2,Bangkok Cuisine,"[10718 70th Rd, Forest Hills, NY 11375]",4.0
3,Sup Thai Kitchen,"[178-19 Union Tpke, Fresh Meadows, NY 11366]",4.5
4,Nur Thai,"[63-32 Woodhaven Blvd, Rego Park, NY 11374]",4.0
5,Kitchen 79,"[37-70 79th St, Jackson Heights, NY 11372]",4.5
6,Ayada Thai,"[7708 Woodside Ave, Elmhurst, NY 11373]",4.0
7,Ma Kin,"[7419 Metropolitan Ave, Middle Village, NY 11379]",4.5
8,SriPraPhai Thai Restaurant,"[64-13 39th Ave, Woodside, NY 11377]",4.0
9,Pye Boat Noodle,"[35-13 Broadway, Astoria, NY 11106]",4.5


In [37]:
pd.DataFrame(parsed_results, columns=['Name', 'Address', 'Rating'])

,Name,Address,Rating
0,Kin'd,"[192-03 Union Tpke, Fresh Meadows, NY 11366]",4.5
1,Thai Mission,"[70-11 Grand Ave, Maspeth, NY 11378]",5.0
2,Bangkok Cuisine,"[10718 70th Rd, Forest Hills, NY 11375]",4.0
3,Sup Thai Kitchen,"[178-19 Union Tpke, Fresh Meadows, NY 11366]",4.5
4,Nur Thai,"[63-32 Woodhaven Blvd, Rego Park, NY 11374]",4.0
5,Kitchen 79,"[37-70 79th St, Jackson Heights, NY 11372]",4.5
6,Ayada Thai,"[7708 Woodside Ave, Elmhurst, NY 11373]",4.0
7,Ma Kin,"[7419 Metropolitan Ave, Middle Village, NY 11379]",4.5
8,SriPraPhai Thai Restaurant,"[64-13 39th Ave, Woodside, NY 11377]",4.0
9,Pye Boat Noodle,"[35-13 Broadway, Astoria, NY 11106]",4.5
